### clone代码

In [ ]:
!git clone https://github.com/modelscope/modelscope-agent.git

### 安装特定依赖

In [ ]:
! cd modelscope-agent && ! pip install -r requirements.txt
! pip install transformers_stream_generator
! pip install invisible-watermark>=2.0
! pip install accelerate --upgrade


In [ ]:
import os
os.chdir('modelscope-agent/demo')


import sys
sys.path.append('../')

### 中枢大模型启动

In [ ]:
from modelscope_agent.llm import LLMFactory
from modelscope.hub.api import HubApi

model_name = 'modelscope-agent-7b'
model_cfg = {
    'modelscope-agent-7b':{
        'type': 'modelscope',
        'model_id': 'damo/ModelScope-Agent-7B',
        'model_revision': 'v1.0.0',
        'use_raw_generation_config': True,
        'custom_chat': True
    }
}


llm = LLMFactory.build_llm(model_name, model_cfg)


### 新工具注册和Agent构建

In [ ]:
from modelscope_agent.agent import AgentExecutor
from modelscope_agent.tools import Tool

class AliyunRenewInstanceTool(Tool):
    description = '续费一台包年包月ECS实例'
    name = 'RenewInstance'
    parameters: list = [{
        'name': 'instance_id',
        'description': 'ECS实例ID',
        'required': True
    }, {
        'name': 'period',
        'description': '续费时长以月为单位',
        'required': True
    }]

    def _local_call(self, *args, **kwargs):
        instance_id = kwargs['instance_id']
        period = kwargs['period']
        return {'result': f'已完成ECS实例ID为{instance_id}的续费，续费时长{period}月'}

# 新增工具可参考AliyunRenewInstanceTool构建方式，配置相关的api name，description等
additional_tool_list = {
    'RenewInstance': AliyunRenewInstanceTool()
}
# 构建Agent，需要传入llm，工具配置config以及工具检索

agent = AgentExecutor(llm, additional_tool_list=additional_tool_list, tool_retrieval=False)
available_tool_list = ['RenewInstance']
agent.set_available_tools(available_tool_list)

In [ ]:
# 重置对话，清空对话历史
agent.reset()

# remote=True为调用modelscope api，该服务免费支持QPS较低，建议部署在本地，将remote=False
agent.run("请帮我续费一台ECS实例，实例id是：i-rj90a7e840y5cde，续费时长10个月", remote=False, print_info=True)